# Quickstart: using Azure Language Detection client library and REST API

https://learn.microsoft.com/en-us/azure/ai-services/language-service/language-detection/quickstart?tabs=linux%2Cfoundry-classic&pivots=programming-language-python

## Install the client library

In [3]:
pip install azure-ai-textanalytics==5.2.0

185.30s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 952.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 3.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Code example :Sample analyze action

In [6]:
# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------

"""
FILE: sample_analyze_actions.py

DESCRIPTION:
    This sample demonstrates how to submit a collection of text documents for analysis, which consists of a variety
    of text analysis actions, such as Entity Recognition, PII Entity Recognition, Linked Entity Recognition,
    Sentiment Analysis, Key Phrase Extraction, and more.
    The response will contain results from each of the individual actions specified in the request.

USAGE:
    python sample_analyze_actions.py

    Set the environment variables with your own values before running the sample:
    1) AZURE_LANGUAGE_ENDPOINT - the endpoint to your Language resource.
    2) AZURE_LANGUAGE_KEY - your Language subscription key
"""


def sample_analyze_actions() -> None:
    # [START analyze]
    import os
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import (
        TextAnalyticsClient,
        RecognizeEntitiesAction,
        RecognizeLinkedEntitiesAction,
        RecognizePiiEntitiesAction,
        ExtractKeyPhrasesAction,
        AnalyzeSentimentAction,
    )

    endpoint = 'https://luisdemostration.cognitiveservices.azure.com/'
    key = 'CRrztP8azNcFlR4j8XuZVw4e1rI7zlywtl9S7V6eVLQ6ExsdN1lbJQQJ99BKACYeBjFXJ3w3AAAaACOGdMB9'

    text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )

    documents = [
        'We went to Contoso Steakhouse located at midtown NYC last week for a dinner party, and we adore the spot! '
        'They provide marvelous food and they have a great menu. The chief cook happens to be the owner (I think his name is John Doe) '
        'and he is super nice, coming out of the kitchen and greeted us all.'
        ,

        'We enjoyed very much dining in the place! '
        'The Sirloin steak I ordered was tender and juicy, and the place was impeccably clean. You can even pre-order from their '
        'online menu at www.contososteakhouse.com, call 312-555-0176 or send email to order@contososteakhouse.com! '
        'The only complaint I have is the food didn\'t come fast enough. Overall I highly recommend it!'
    ]

    poller = text_analytics_client.begin_analyze_actions(
        documents,
        display_name="Sample Text Analysis",
        actions=[
            RecognizeEntitiesAction(),
            RecognizePiiEntitiesAction(),
            ExtractKeyPhrasesAction(),
            RecognizeLinkedEntitiesAction(),
            AnalyzeSentimentAction(),
        ],
    )

    document_results = poller.result()
    for doc, action_results in zip(documents, document_results):
        print(f"\nDocument text: {doc}")
        for result in action_results:
            if result.kind == "EntityRecognition":
                print("...Results of Recognize Entities Action:")
                for entity in result.entities:
                    print(f"......Entity: {entity.text}")
                    print(f".........Category: {entity.category}")
                    print(f".........Confidence Score: {entity.confidence_score}")
                    print(f".........Offset: {entity.offset}")

            elif result.kind == "PiiEntityRecognition":
                print("...Results of Recognize PII Entities action:")
                for pii_entity in result.entities:
                    print(f"......Entity: {pii_entity.text}")
                    print(f".........Category: {pii_entity.category}")
                    print(f".........Confidence Score: {pii_entity.confidence_score}")

            elif result.kind == "KeyPhraseExtraction":
                print("...Results of Extract Key Phrases action:")
                print(f"......Key Phrases: {result.key_phrases}")

            elif result.kind == "EntityLinking":
                print("...Results of Recognize Linked Entities action:")
                for linked_entity in result.entities:
                    print(f"......Entity name: {linked_entity.name}")
                    print(f".........Data source: {linked_entity.data_source}")
                    print(f".........Data source language: {linked_entity.language}")
                    print(
                        f".........Data source entity ID: {linked_entity.data_source_entity_id}"
                    )
                    print(f".........Data source URL: {linked_entity.url}")
                    print(".........Document matches:")
                    for match in linked_entity.matches:
                        print(f"............Match text: {match.text}")
                        print(f"............Confidence Score: {match.confidence_score}")
                        print(f"............Offset: {match.offset}")
                        print(f"............Length: {match.length}")

            elif result.kind == "SentimentAnalysis":
                print("...Results of Analyze Sentiment action:")
                print(f"......Overall sentiment: {result.sentiment}")
                print(
                    f"......Scores: positive={result.confidence_scores.positive}; \
                    neutral={result.confidence_scores.neutral}; \
                    negative={result.confidence_scores.negative} \n"
                )

            elif result.is_error is True:
                print(
                    f"...Is an error with code '{result.error.code}' and message '{result.error.message}'"
                )

        print("------------------------------------------")

    # [END analyze]


if __name__ == "__main__":
    sample_analyze_actions()


Document text: We went to Contoso Steakhouse located at midtown NYC last week for a dinner party, and we adore the spot! They provide marvelous food and they have a great menu. The chief cook happens to be the owner (I think his name is John Doe) and he is super nice, coming out of the kitchen and greeted us all.
...Results of Recognize Entities Action:
......Entity: Contoso Steakhouse
.........Category: Location
.........Confidence Score: 0.99
.........Offset: 11
......Entity: NYC
.........Category: Location
.........Confidence Score: 1.0
.........Offset: 49
......Entity: last week
.........Category: DateTime
.........Confidence Score: 1.0
.........Offset: 53
......Entity: dinner party
.........Category: Event
.........Confidence Score: 0.85
.........Offset: 69
......Entity: food
.........Category: Product
.........Confidence Score: 0.91
.........Offset: 129
......Entity: chief cook
.........Category: PersonType
.........Confidence Score: 0.56
.........Offset: 166
......Entity: owner

## Sample code 2 :Sample abstract summary

In [16]:
!pip install azure-ai-textanalytics==5.2.0

15669.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [17]:
# This example requires environment variables named "LANGUAGE_KEY" and "LANGUAGE_ENDPOINT"
language_key = 'CRrztP8azNcFlR4j8XuZVw4e1rI7zlywtl9S7V6eVLQ6ExsdN1lbJQQJ99BKACYeBjFXJ3w3AAAaACOGdMB9'
language_endpoint = 'https://luisdemostration.cognitiveservices.azure.com/'

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=language_endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example method for detecting sensitive information (PII) from text 
def pii_recognition_example(client):
    documents = [
        "The employee's SSN is 859-98-0987.",
        "The employee's phone number is 555-555-5555."
    ]
    response = client.recognize_pii_entities(documents, language="en")
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        print("Redacted Text: {}".format(doc.redacted_text))
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))
pii_recognition_example(client)

Redacted Text: The ********'s SSN is ***********.
Entity: employee
	Category: PersonType
	Confidence Score: 0.98
	Offset: 4
	Length: 8
Entity: 859-98-0987
	Category: USSocialSecurityNumber
	Confidence Score: 0.85
	Offset: 22
	Length: 11
Redacted Text: The ********'s phone number is ************.
Entity: employee
	Category: PersonType
	Confidence Score: 0.99
	Offset: 4
	Length: 8
Entity: 555-555-5555
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 31
	Length: 12
